In [ ]:
# !pip install fastapi uvicorn nest_asyncio

In [2]:
PORT = 5000

In [6]:
import uvicorn
from fastapi import FastAPI
import nest_asyncio
from pydantic import BaseModel

nest_asyncio.apply()

app = FastAPI()

class AnalyzeRequest(BaseModel):
    content: str

@app.get("/health")
def health_check():
    return {"status": "healthy"}

@app.post("/analyze")
def analyze(request: AnalyzeRequest):
    return {"received_content": request.content}

uvicorn.run(app, host="0.0.0.0", port=PORT)

INFO:     Started server process [29352]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [29352]
